In [1]:
import socket
import asyncio
from bleak import BleakScanner, BleakClient
import argparse
import json
import re

from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
from datetime import datetime
import ast

CHARACTERISTIC_UUID = 'f22535de-5375-44bd-8ca9-d0ea9ff9e410'
CONTROL_UNIT_NAME = 'QT Py ESP32-S3'
HOST = "127.0.0.1"  # Standard loopback interface address (localhost)
PORT = 9051  # Port to listen on (non-privileged ports are > 1023)


import Python_Unity_Server as pus

In [7]:
app = Flask(__name__)
CORS(app)

data_manager = pus.DataManager()

@app.route('/endpoint', methods=['POST'])
def endpoint():
    data = request.get_json()
    if not data:
        return jsonify({'success': False, 'error': 'No JSON data provided'}), 400

    if 'start' in data:
        data_manager.start()
        return jsonify({'started': True}), 200
    elif 'quit' in data:
        data_manager.quit()
        return jsonify({'quit': True}), 200
    elif 'data' in data:
        success = data_manager.add_data(data)
        return jsonify({'success': success}), 200
    else:
        return jsonify({'success': False, 'error': 'Invalid data'}), 400

@app.route('/data', methods=['GET'])
def get_data():
    return jsonify(data_manager.get_data())

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [ ]:
if __name__ == "__main__":
    # Initialize the argument parser
    parser = argparse.ArgumentParser(description="Read CHARACTERISTIC_UUID and CONTROL_UNIT_NAME from the command line.")

    # Add arguments with flags
    parser.add_argument(
        "-uuid", "--characteristic_uuid", required=False, type=str,
        default="f22535de-5375-44bd-8ca9-d0ea9ff9e410",
        help="The UUID of the characteristic"
    )
    
    parser.add_argument(
        "-name", "--control_unit_name", required=False, type=str, 
        default="QT Py ESP32-S3",
        help="The Bluetooth name of the control unit"
    )

    parser.add_argument(
        "-host", "--host", required=False, type=str,
        default="127.0.0.1",
        help="The host IP address"
    )

    parser.add_argument(
        "-port", "--port", required=False, type=int,
        default=9051,
        help="The port number"
    )

    # Parse the arguments
    args, props = parser.parse_known_args()

    # Access and print the parameters
    print(f"CHARACTERISTIC_UUID: {args.characteristic_uuid}")
    print(f"CONTROL_UNIT_NAME: {args.control_unit_name}")
    print(f"HOST: {args.host}")
    print(f"PORT: {args.port}")

    CHARACTERISTIC_UUID = args.characteristic_uuid
    CONTROL_UNIT_NAME = args.control_unit_name
    HOST = args.host
    PORT = args.port

    
    while True:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.bind((HOST, PORT))
            print('python server start listening')
            s.listen()
            conn, addr = s.accept()
            with conn:
                print(f"TCP socket connected by {addr}")
                # while True:
                    # data = conn.recv(1024)
                    # print('TCP data recv = ', data)
                    # if not data:
                    #     break
                asyncio.create_task(pus.main(conn))
        print('TCP socket disconnected! restart now...')


CHARACTERISTIC_UUID: f22535de-5375-44bd-8ca9-d0ea9ff9e410
CONTROL_UNIT_NAME: QT Py ESP32-S3
HOST: 127.0.0.1
PORT: 9051
python server start listening
TCP socket connected by ('127.0.0.1', 64677)
TCP socket disconnected! restart now...
python server start listening
TCP socket connected by ('127.0.0.1', 64932)
TCP socket disconnected! restart now...
python server start listening


In [9]:
import socket
import asyncio
from bleak import BleakScanner, BleakClient
import argparse
import json
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
from datetime import datetime
import ast
import threading

# Define constants
CHARACTERISTIC_UUID = 'f22535de-5375-44bd-8ca9-d0ea9ff9e410'
CONTROL_UNIT_NAME = 'QT Py ESP32-S3'
HOST = "127.0.0.1"
PORT = 9051

# Flask app setup
app = Flask(__name__)
CORS(app)

# DataManager class to handle data operations
class DataManager:
    def __init__(self):
        self.data_rows = []
        self.started = False

    def start(self):
        self.started = True
        self.data_rows = []

    def quit(self):
        self.started = False
        df = pd.DataFrame(self.data_rows)
        df.to_csv('data.csv', index=False)

    def add_data(self, data):
        if self.started:
            row_data = handle_json(data)
            self.data_rows.append(row_data)
            return True
        else:
            return False

    def get_data(self):
        return self.data_rows

data_manager = DataManager()

def handle_json(json_data):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    row_data = {"timestamp": timestamp}

    data_items = json_data.get("data", [])
    for item in data_items:
        name = item.get("name")
        value = item.get("value")

        if name is not None:
            if isinstance(value, str):
                try:
                    parsed_value = ast.literal_eval(value)
                    row_data[name] = parsed_value
                except (ValueError, SyntaxError):
                    row_data[name] = value
            else:
                row_data[name] = value

    return row_data

# Bluetooth command creation
def create_command(addr, mode, duty, freq):
    serial_group = addr // 30
    serial_addr = addr % 30
    byte1 = (serial_group << 2) | (mode & 0x01)
    byte2 = 0x40 | (serial_addr & 0x3F)  # 0x40 represents the leading '01'
    byte3 = 0x80 | ((duty & 0x0F) << 3) | (freq)  # 0x80 represents the leading '1'
    return bytearray([byte1, byte2, byte3])

async def set_motor(client, command_data):
    data_chunks = [command_data[i:i + 10] for i in range(0, len(command_data), 10)]
    for data_chunk in data_chunks:
        command = bytearray()
        for data_segment in data_chunk:
            command += create_command(data_segment['addr'], data_segment['mode'], data_segment['duty'], data_segment['freq'])
        command += bytearray([0xFF, 0xFF, 0xFF]) * (20 - len(data_chunk))
        await client.write_gatt_char(CHARACTERISTIC_UUID, command)

async def handle_bluetooth(socket_conn):
    devices = await BleakScanner.discover()
    for d in devices:
        if d.name == CONTROL_UNIT_NAME:
            async with BleakClient(d.address) as client:
                while True:
                    data = socket_conn.recv(1024)
                    if not data:
                        break
                    data_str = data.decode('utf-8').strip()
                    command_data = json.loads(data_str)
                    await set_motor(client, command_data)

# TCP Server to handle both protocols
def tcp_server():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()
        print(f"TCP server listening on {HOST}:{PORT}")
        conn, addr = s.accept()
        with conn:
            print(f"Connected by {addr}")
            while True:
                data = conn.recv(1024)
                if not data:
                    break
                data_str = data.decode('utf-8').strip()
                print(f"Received: {data_str}")
                try:
                    request = json.loads(data_str)
                    if 'bluetooth_command' in request:
                        print("Bluetooth command received")
                        # Handle Bluetooth commands
                        asyncio.run(handle_bluetooth(conn))
                    elif 'api_command' in request:
                        print("API command received")
                        # Handle API commands
                        handle_api_commands(request['api_command'], conn)
                    else:
                        print("Invalid command received")
                        asyncio.run(handle_bluetooth(conn))
                except json.JSONDecodeError:
                    print("Invalid JSON received")

            print(f"Connection with {addr} closed")

# Flask API commands
def handle_api_commands(command, conn):
    response = {}
    if command == "start":
        data_manager.start()
        response = {'started': True}
    elif command == "quit":
        data_manager.quit()
        response = {'quit': True}
    elif "data" in command:
        success = data_manager.add_data(command)
        response = {'success': success}
    elif command == "get_data":
        response = data_manager.get_data()
    conn.sendall(json.dumps(response).encode('utf-8'))

# Run the Flask app in a separate thread
def run_flask():
    app.run(port=5000)

if __name__ == "__main__":
    # Start Flask server in a separate thread
    flask_thread = threading.Thread(target=run_flask)
    flask_thread.start()
    
    # Start the TCP server
    tcp_server()


TCP server listening on 127.0.0.1:9051
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Connected by ('127.0.0.1', 53587)
Received: {"addr": 1, "mode": 1, "duty": 7, "freq": 2}
Invalid command received


RuntimeError: asyncio.run() cannot be called from a running event loop

In [14]:
import socket
import asyncio
from bleak import BleakScanner, BleakClient
import argparse
import json

CHARACTERISTIC_UUID = 'f22535de-5375-44bd-8ca9-d0ea9ff9e410'
CONTROL_UNIT_NAME = 'QT Py ESP32-S3'
HOST = "127.0.0.1"  # Standard loopback interface address (localhost)
PORT = 9051  # Port to listen on (non-privileged ports are > 1023)

def create_command(addr, mode, duty, freq):
    serial_group = addr // 30
    serial_addr = addr % 30
    byte1 = (serial_group << 2) | (mode & 0x01)
    byte2 = 0x40 | (serial_addr & 0x3F)  # 0x40 represents the leading '01'
    byte3 = 0x80 | ((duty & 0x0F) << 3) | (freq)  # 0x80 represents the leading '1'
    return bytearray([byte1, byte2, byte3])

async def setMotor(client, socket_conn):
    while True:
        data = socket_conn.recv(1024)
        if not data:
            break
        print('TCP data recv = ', data)
        data_str = data.decode('utf-8').strip()
        data_segments = data_str.split("\n")
        # create data chunks of 10
        data_chunks = [data_segments[i:i + 10] for i in range(0, len(data_segments), 10)]
        for data_chunk in data_chunks:
            command = bytearray([])
            for data_segment in data_chunk:
                data_parsed = json.loads(data_segment, encoding='utf-8')
                command = command + create_command(data_parsed['addr'], data_parsed['mode'], data_parsed['duty'], data_parsed['freq'])
            command = command + bytearray([0xFF, 0xFF, 0xFF]) * (20-len(data_chunk))
            await client.write_gatt_char(CHARACTERISTIC_UUID, command)

async def main(socket_conn):
    devices = await BleakScanner.discover()
    for d in devices:
        print('device name = ', d.name)
        if d.name != None:
            if d.name == CONTROL_UNIT_NAME:
                print('feather device found!!!')
                async with BleakClient(d.address) as client:
                    print(f'BLE connected to {d.address}')
                    val = await client.read_gatt_char(CHARACTERISTIC_UUID)
                    print('Motor read = ', val)
                    while True:
                        await setMotor(client, socket_conn)

if __name__ == "__main__":
    # Initialize the argument parser
    parser = argparse.ArgumentParser(description="Read CHARACTERISTIC_UUID and CONTROL_UNIT_NAME from the command line.")

    # Add arguments with flags
    parser.add_argument(
        "-uuid", "--characteristic_uuid", required=False, type=str,
        default="f22535de-5375-44bd-8ca9-d0ea9ff9e410",
        help="The UUID of the characteristic"
    )
    parser.add_argument(
        "-name", "--control_unit_name", required=False, type=str, 
        default="QT Py ESP32-S3",
        help="The Bluetooth name of the control unit"
    )

    parser.add_argument(
        "-host", "--host", required=False, type=str,
        default="127.0.0.1",
        help="The host IP address"
    )

    parser.add_argument(
        "-port", "--port", required=False, type=int,
        default=9051,
        help="The port number"
    )

    # Parse the arguments
    args = parser.parse_args()

    # Access and print the parameters
    print(f"CHARACTERISTIC_UUID: {args.characteristic_uuid}")
    print(f"CONTROL_UNIT_NAME: {args.control_unit_name}")
    print(f"HOST: {args.host}")
    print(f"PORT: {args.port}")

    CHARACTERISTIC_UUID = args.characteristic_uuid
    CONTROL_UNIT_NAME = args.control_unit_name
    HOST = args.host
    PORT = args.port
    
    while True:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.bind((HOST, PORT))
            print('python server start listening')
            s.listen()
            conn, addr = s.accept()
            with conn:
                print(f"TCP socket connected by {addr}")
                # while True:
                    # data = conn.recv(1024)
                    # print('TCP data recv = ', data)
                    # if not data:
                    #     break
                asyncio.run(main(conn))
        print('TCP socket disconnected! restart now...')

usage: ipykernel_launcher.py [-h] [-uuid CHARACTERISTIC_UUID]
                             [-name CONTROL_UNIT_NAME] [-host HOST]
                             [-port PORT]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\Pablo\AppData\Roaming\jupyter\runtime\kernel-v36a7ccbf7eb8d07735cc55c031db9e19b4aba2fbc.json


SystemExit: 2